## Xarray

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

### Trabajando con un archivo netCDF

In [ ]:
ds = xr.open_dataset('pgbh00.gdas.2008081512.nc')

In [ ]:
ds

In [ ]:
# accede a coordenadas
ds.lons

In [ ]:
ds.lats

In [ ]:
ds.pres

In [ ]:
ds.time

In [ ]:
# accede a variables
ds.pwat

In [ ]:
ds.pwat.plot()

In [ ]:
plt.show()

In [ ]:
ds.pwat.plot(cmap='RdBu')
plt.show()

In [ ]:
ds.pwat.plot(cmap='RdBu',vmin=0,vmax=50)
plt.show()

In [ ]:
# arreglemos las coordenadas de longitud
ds['lons'] = ds['lons'] - 360

In [ ]:
ds.pwat.plot(cmap='RdBu',vmin=0,vmax=50)
plt.show()

** creamos diccionario con opciones de configuración **

In [ ]:
figsetup = dict(cmap='RdBu',vmin=0,vmax=50,extend='neither')

In [ ]:
ds.pwat.plot(**figsetup)
plt.show()

### Trabajando con múltiples archivos netCDF

In [ ]:
from glob import glob

In [ ]:
ncfiles = glob('*.nc')

In [ ]:
ncfiles

In [ ]:
first = True
for nc in ncfiles:
    if first:
        ds_all = xr.open_dataset(nc)
        first = False
    else:
        ds = xr.open_dataset(nc)
        ds_all = xr.concat([ds_all,ds], dim='time')

In [ ]:
ds_all



**Para seleccionar el índice de la dimensión: `isel`**

In [ ]:
ds_all.pwat.isel(time=3).plot(**figsetup)
plt.show()

In [ ]:
# arreglamos valores lons
ds_all['lons'] -= 360

In [ ]:
ds_all.pwat.isel(time=3).plot(**figsetup)
plt.show()

**Para seleccionar el valor de la dimensión: `sel`**

In [ ]:
ds_all.time

In [ ]:
ds_all.pwat.sel(time='2008-08-15 12:00').plot(**figsetup)
plt.show()

**Podemos utilizar `sel` para hacer la figura de cada paso de tiempo**

In [ ]:
for time in ds_all.time:
    fig, ax = plt.subplots()
    figsetup['ax'] = ax
    ds_all.pwat.sel(time=time).plot(**figsetup)
plt.show()

** O Podemos utilizar el metodo faceting de xarray**

In [ ]:
figsetup = dict(cmap='RdBu',vmin=0,vmax=50,extend='neither')
figsetup['x'] = 'lons'
figsetup['y'] = 'lats'
figsetup['col'] = 'time'  # <- faceting
figsetup['col_wrap'] = 2

In [ ]:
figsetup

In [ ]:
ds_all.pwat.plot(**figsetup)
plt.show()

In [ ]:
ds_all.pwat.plot.contourf(**figsetup)
plt.show()

In [ ]:
ds_all.pwat.plot.contour(**figsetup)
plt.show()

### Agregando lineas costeras y limites geopolíticos

In [ ]:
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
figsetup

In [ ]:
figsetup['transform'] = ccrs.PlateCarree()
figsetup['subplot_kws'] = {'projection': ccrs.PlateCarree()}

In [ ]:
figsetup

In [ ]:
ds_all.pwat.plot(**figsetup)
plt.show()

In [ ]:
plot_handle = ds_all.pwat.plot(**figsetup);

# opciones para la grilla
gl_setup = dict(crs=ccrs.PlateCarree(), 
                  draw_labels=False,
                  linewidth=2, 
                  color='gray', 
                  alpha=0.5, 
                  linestyle='--')

for ax in plot_handle.axes.flat:
    ax.add_feature(cartopy.feature.BORDERS)
    ax.add_feature(cartopy.feature.COASTLINE)
    ax.set_extent([-100, -65, -45, -15]) # utilizar extent en lugar de xlim/ylim
    ax.set_aspect('equal','box-forced')  # <- obligatorio
    
    # formato de grilla lat/lon
    gl = ax.gridlines(**gl_setup)
    gl.xlabels_bottom = True
    gl.ylabels_left = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
plt.show()

** usar `enumerate` para arreglar axis individuales **

In [ ]:
plot_handle = ds_all.pwat.plot(**figsetup);

gl_setup = dict(crs=ccrs.PlateCarree(), 
              draw_labels=False,
              linewidth=2, 
              color='gray', 
              alpha=0.5, 
              linestyle='--')

for n, ax in enumerate(plot_handle.axes.flat):  # <- enumerate
    ax.add_feature(cartopy.feature.BORDERS)
    ax.add_feature(cartopy.feature.COASTLINE)
    ax.set_extent([-100, -65, -45, -15]) # utilizar extent en lugar de xlim/ylim
    ax.set_aspect('equal','box-forced')  # <- obligatorio
    
    # formato de grilla lat/lon
    gl = ax.gridlines(**gl_setup)
    if n in [0,2]:
        gl.ylabels_left = True
    elif n == 4:
        gl.xlabels_bottom = True
        gl.ylabels_left = True            
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

plt.show()